# 尝试写一个整形权重CNN的预测

In [1]:
%pdb 1
from __future__ import print_function
import numpy as np
import struct

import time
import pickle
import os

Automatic pdb calling has been turned ON


In [2]:
np.random.seed(1337)  # for reproducibility
nb_classes = 34
nb_epoch = 30
batch_size = 128

image_higth, image_width = 15, 15
# lr = [0.05, 0.01, 0.005]
layer1 = 10
hidden1 = 40
region = 3

# 模型权重根目录
model_root_path = '/home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/modelAndData1122/'
model_file_list_path = os.listdir(os.path.join(model_root_path, 'model'))


定义必要的函数

In [3]:
%%time
def load_valdata(version='1122'):
    """读取不同版本的测试集
    1120 - 对应 /home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/modelAndData1120/Data/
    1122 - 对应 /home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/modelAndData1122/Data/

    :param dir_path: str
    :param version: str
    :return:
    """

    # 数据集根目录
    data_root_path = '/home/jdwang/PycharmProjects/digitRecognition/int_weight_predict/'

    # 读取验证数据、测试数据
    if version == '1122':
        val_file_path = os.path.join(data_root_path, 'modelAndData1122/Data/', 'TrainSet_trainAndVal_testSet.pickle')
        other_file_path = os.path.join(data_root_path, 'modelAndData1122/Data/', 'Olddata_TestSet.pickle')
        with open(val_file_path, 'rb') as train_file:
            train_X = pickle.load(train_file)
            train_y = pickle.load(train_file)
            val_X, val_y = None, None
            test_X = pickle.load(train_file)
            test_y = pickle.load(train_file)
    elif version == '1120':
        val_file_path = os.path.join(data_root_path, 'modelAndData1120/Data/', 'valSet&TestSet.pickle')
        other_file_path = os.path.join(data_root_path, 'modelAndData1120/Data/', 'Olddata_TestSet.pickle')
        with open(val_file_path, 'rb') as train_file:
            val_X = pickle.load(train_file)
            val_y = pickle.load(train_file)
            test_X = pickle.load(train_file)
            test_y = pickle.load(train_file)
    else:
        raise NotImplementedError

    with open(other_file_path, 'rb') as otherFile:
        other_X = pickle.load(otherFile)
        other_y = pickle.load(otherFile)

    return (val_X, val_y), (test_X, test_y), (other_X, other_y)


def Net_model(layer1, hidden1, region, rows, cols, nb_classes, lr=0.01, decay=1e-6, momentum=0.9):
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation, Flatten
    from keras.layers.convolutional import Convolution2D, MaxPooling2D
    from keras.optimizers import SGD
    from keras import backend as K
    model = Sequential()

    model.add(Convolution2D(layer1, region, region,
                            border_mode='valid',
                            input_shape=(1, rows, cols)))

    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # 平铺

    model.add(Dense(hidden1))  # Full connection 1:  1000
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    # 获取CNN的中间结果
    mid_output = K.function(inputs=[
        model.layers[0].input,
        K.learning_phase(),
    ],
        outputs=[
            model.layers[-9].output,
            model.layers[-8].output,
            model.layers[-7].output,
            model.layers[-6].output,
            model.layers[-5].output,
            model.layers[-4].output,
            model.layers[-3].output,
            model.layers[-2].output,
            model.layers[-1].output,
        ]
    )

    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

    return model, mid_output


def test_model(model_file, X_test, Y_test):
    # 加载模型架构
    # 这里的 lr 设置什么不影响
    model, mid_output = Net_model(layer1, hidden1, region, image_higth, image_width, nb_classes=34, lr=0)
    model.load_weights(model_file)

    # 预测
    predicted = model.predict_classes(X_test, verbose=0)

    return count_result(predicted, Y_test)


def count_result(predicted, Y_test):
    '''统计预测情况，包括混淆集个数等

    :return:
    '''
    # 统计混淆集
    badcase = {}
    for i in range(0, len(Y_test)):
        if (tramsform(predicted[i]) in ['1', 'I'] and tramsform(Y_test[i]) in ['1', 'I']):
            # 1跟I区分，只要 是 测试 成1或I，而实际值是 1或I都算对
            predicted[i] = Y_test[i]
        if predicted[i] != Y_test[i]:
            ch1 = tramsform(Y_test[i])
            ch2 = tramsform(predicted[i])
            string = ','.join(sorted([ch1, ch2]))

            if badcase.has_key(string):
                badcase[string] += 1
            else:
                badcase[string] = 1

    # 计算测试准确率
    test_accuracy = np.mean(np.equal(predicted, Y_test))
    graterThan2 = 0
    graterThan5 = 0
    graterThan10 = 0
    for key, value in badcase.items():
        if value >= 2:
            graterThan2 += 1
        if value >= 5:
            graterThan5 += 1
        if value >= 10:
            graterThan10 += 1

    return test_accuracy, graterThan2, graterThan5, graterThan10, badcase


def tramsform(num):
    """ 数字 转换成 字符

    :param num:
    :return:
    """
    if num < 10:
        return str(num)
    else:
        if num >= 24:
            num += 1
        if num >= 31:
            num += 1
        return chr(ord('A') + num - 10)


def conv_pool_operation(img, conv_W, conv_b):
    """CNN 卷积 和 池化操作
        一张图片
    :param img: array-3D
        一张图片，3D，(num_of_channels,img_height,img_width)
    :param conv_W: array-4D
        10*3*3
    :param conv_b:
    :return: array-3D
    """
    filter_row, filter_col = conv_W.shape[2:]
    img_row, img_col = img.shape[1:]
    # convolution
    # 3D
    conv_result = np.zeros((conv_W.shape[0], img_row - filter_row + 1, img_col - filter_col + 1))
    # quit()
    for filter_index in range(conv_W.shape[0]):
        j = conv_W[filter_index, 0].flatten()[-1::-1]
        for x in range(0, img_row - filter_row + 1):
            for y in range(0, img_col - filter_col + 1):
#                 i = img[0, x:x + filter_row, y:y + filter_col].flatten()
#                 
                conv_result[filter_index,
                            x,
                            y] = tanh_approximate_function(
                    np.dot(
                        img[0, x:x + filter_row, y:y + filter_col].flatten(), 
                        j
                    ) 
                    + conv_b[filter_index]
                )
                # print(np.dot(i, j) + conv_b[filter_index])

    # conv_result = tanh_approximate_function(conv_result)
    pool_row, pool_col = 2, 2
    pool_result = np.zeros(
        (conv_result.shape[0], conv_result.shape[1] / pool_row, conv_result.shape[2] / pool_col))
    # max-pooling
    for channel_index in range(pool_result.shape[0]):
        for x in range(pool_result.shape[1]):
            for y in range(pool_result.shape[2]):
                pool_result[
                    channel_index,
                    x,
                    y
                ] = np.max(
                    conv_result[
                    channel_index,
                    x * pool_row: (x + 1) * pool_row,
                    y * pool_col: (y + 1) * pool_col
                    ])
    return pool_result


def hidden_operation(feature_vector, W, b, activion='tanh'):
    """ 隐含层操作

    :param activion: str
        激活函数
    :param feature_vector: array-like
        一张 图片的 feature vector，1D
    :param W: array-2D
        权重
    :param b: array-1D
        偏差
    :return:
    """

    if activion == 'tanh':
        # return np.asarray([tanh_approximate_function(item) for item in temp])
        return map(lambda x: tanh_approximate_function(np.dot(feature_vector, x[0]) + x[1]), zip(W.transpose(), b))
    elif activion == 'none':
        # return temp
        return map(lambda x:np.dot(feature_vector, x[0])+x[1], zip(W.transpose(),b))
    else:
        raise NotImplementedError


def tanh_approximate_function(x):
    """
         * tanh x=sinh x / cosh x
         * 其中sinh x=(e^(x)-e^(-x))/2 ，cosh x=(e^x+e^(-x))/2
         * 所以tanhx = (e^(x)-e^(-x)) /(e^x+e^(-x))
    """
    #     return (int)(tanh(x));

    if x > 10:
        return 1
    elif x < -10:
        return -1
    else:
        return (int)((np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x)))


def cnn_batch_predict(X_val, weights):
    '''CNN批量预测

    :param X_val:
    :param weights:
    :return:
    '''
    weights = [(item * 1e6).astype(dtype=int) for item in weights]
    # result = cnn_predict(X_val[0],weights)
    result = []
    for index, img in enumerate(X_val):
        if (index + 1) % 1000 == 0:
            print('%d' % (index + 1))
        result.append(cnn_predict(img, weights))
    return np.asarray(result)


def cnn_predict(img, weights):
    '''单张图片的预测

    :param img:
    :param weights:
    :return:
    '''
    # 3D
    %time imgs_conv_result = conv_pool_operation(img, weights[0], weights[1])
    # print('conv over..')
    %time flatten_result = imgs_conv_result.flatten()
    # print('flatten over..')
    # print(flatten_result.shape)
    %time hidden1_output = hidden_operation(flatten_result, weights[2], weights[3], activion='tanh')

    # print('hidden1 over..')
    %time hidden2_output = hidden_operation(hidden1_output, weights[4], weights[5], activion='none')
    # print('hidden2 over..')
    # print(hidden2_output.shape)
    result = np.argmax(hidden2_output)
    return result



def cnn_batch_predict_spark(X_val_rdd,y_val, weights):
    weights = [(item * 1e5).astype(dtype=int) for item in weights]
#     result_rdd = X_val_rdd.map(lambda x:cnn_predict(x,weights))
    
    result_rdd = X_val_rdd.map(lambda x:conv_pool_operation(x, weights[0], weights[1]))\
    .map(lambda x: x.flatten())\
    .map(lambda x:hidden_operation(x,weights[2], weights[3],activion='tanh'))\
    .map(lambda x:hidden_operation(x,weights[4], weights[5],activion='none'))\
    .map(np.argmax)
    
# #     result_rdd.cache()
    result = result_rdd.collect()
#     accu = np.mean(np.asarray(result) == y_val) 
# #     print(accu)
    return result,count_result(result,y_val)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs


In [4]:
%%time
# region 读取数据集：验证数据(64369个)、测试数据(64381个)、其他应用数据集(243391个)
(X_val, y_val), (X_test, y_test), (X_other, y_other) = load_valdata(version='1122')

print(X_test.shape)
print(X_other.shape)

(64381, 1, 15, 15)
(243391, 1, 15, 15)
CPU times: user 3.39 s, sys: 268 ms, total: 3.66 s
Wall time: 3.77 s


### 将图片数据序列化

In [5]:
%%time
# 测试集
X_val_rdd = sc.parallelize(X_test)
y_val = y_test
X_val_rdd.cache

# 应用集
X_other_rdd = sc.parallelize(X_other)
y_other = y_other

X_other_rdd.cache

CPU times: user 44 ms, sys: 44 ms, total: 88 ms
Wall time: 504 ms


## 保存34分类TOP21的测试集预测结果

In [ ]:
%%time

# 34分类前21个模型的预测结果
predict_result_34class_file = open(os.path.join(model_root_path,'34class_test_predict_result_e5.pkl'),'w')

# TOP21个模型的编号
run_id = [99,129,141,245,249,270,287,300,311,375,425,509,543,630,758,864,875,890,905,975,1014]
run_idx = 0
with open(os.path.join(model_root_path, '34class_model_weight.pkl'), 'r') as fin:
    for index in range(1, len(model_file_list_path) + 1):
        # 从 模型1 开始，依次往后
        # 找到对应模型文件
#         if index != run_id[run_idx]:
#             weights = pickle.load(fin)
#             continue
        
        if index < run_id[run_idx]:
            weights = pickle.load(fin)
            continue

        
        weights = pickle.load(fin)

        # print('OK')
        int_predict,(test_accuracy, graterThan2, graterThan5, graterThan10,badcase) = \
        cnn_batch_predict_spark(X_val_rdd,y_val, weights)
        
        print(index,test_accuracy, graterThan2, graterThan5,graterThan10)
        print(badcase)
        
        pickle.dump(np.asarray(int_predict),
                    predict_result_34class_file
                   )

        
#         print(test_accuracy, graterThan5, graterThan10)
        run_idx += 1
        if run_idx>=len(run_id):
            break
#         break

99 0.999565089079 8 0 0
{'3,8': 2, '7,T': 1, '0,Q': 2, '7,P': 1, '1,7': 3, '1,T': 3, 'C,G': 1, '5,6': 2, '8,B': 1, '3,S': 2, 'X,Y': 1, '4,A': 1, '0,D': 1, '0,C': 1, '5,L': 2, '2,Z': 2, 'G,Q': 1, 'H,K': 1}
129 0.999580621612 6 1 0
{'9,S': 1, '0,Q': 1, 'H,W': 1, 'F,P': 1, '1,T': 2, '5,6': 1, '8,B': 4, '6,G': 4, '5,U': 2, 'D,Q': 1, '0,D': 5, 'A,X': 1, '2,Z': 2, 'C,Q': 1}
141 0.999720414408 6 0 0
{'A,N': 1, '6,K': 1, '0,Q': 2, 'E,F': 2, 'C,G': 1, '5,6': 4, 'F,P': 1, '6,8': 2, '0,D': 2, '2,Z': 2}
245 0.999487426415 10 1 0
{'6,E': 1, '1,X': 1, '0,Q': 5, '1,7': 3, '1,4': 3, 'C,G': 1, '5,6': 2, '0,G': 2, '6,8': 2, '0,D': 3, '0,C': 1, '8,Y': 1, 'F,P': 2, '5,B': 1, '5,S': 1, '1,T': 2, '3,B': 2}
249 0.999658284276 6 0 0
{'9,S': 1, '5,S': 1, '8,B': 1, '5,U': 2, '0,G': 2, '4,A': 4, '0,D': 3, '0,C': 2, '2,Z': 3, '6,F': 1, 'C,Q': 1, '3,9': 1}
270 0.999580621612 7 2 0
{'7,Z': 1, '3,8': 2, '0,Q': 2, '2,E': 2, '1,7': 5, '2,Z': 2, '8,B': 1, '6,F': 1, 'F,H': 1, '0,D': 6, '5,L': 2, '5,S': 1, 'G,Q': 1}
287 

## 整理下badcase混淆集情况

In [ ]:
s='''{'0,6': 4, '1,9': 1, '1,4': 1, '6,8': 7, 'H,K': 2, '5,8': 3, '3,8': 4, '5,6': 8, 'G,Q': 5, '8,L': 1, '9,S': 4, '7,P': 1, '7,T': 1, '8,B': 34, '4,A': 2, '9,H': 5, '0,Q': 7, '1,T': 14, 'C,G': 4, '6,E': 6, '0,D': 30, '0,C': 5, '2,Z': 20, 'B,G': 2, '1,B': 2}
{'D,Q': 2, '4,Q': 1, '6,E': 1, '0,Q': 17, 'B,Z': 2, '1,7': 9, '1,T': 1, '5,6': 6, '8,B': 12, '6,G': 4, '6,8': 4, '4,A': 2, '0,D': 36, '9,B': 2, '2,Z': 1}
{'J,T': 2, '7,Z': 2, '0,4': 1, '3,8': 3, '9,S': 2, '0,Q': 6, '1,Y': 2, 'E,F': 1, '1,7': 1, '1,T': 2, '5,6': 22, '8,B': 11, '6,8': 1, '0,D': 21, '0,C': 5, 'F,P': 1, '5,S': 3, '3,B': 3}
{'1,7': 9, '6,8': 5, '3,8': 3, '4,6': 4, 'F,P': 3, '5,6': 19, '3,B': 7, 'G,Q': 1, '9,S': 2, '7,T': 2, '8,B': 5, '4,A': 3, '5,B': 1, '4,M': 1, '0,Q': 12, '1,Y': 3, 'C,D': 1, '1,W': 2, '1,T': 2, 'C,G': 1, '0,D': 40, '0,C': 1}
{'7,T': 2, '9,P': 2, '0,Q': 7, '6,E': 2, '1,T': 11, '5,6': 4, '8,B': 5, '6,G': 2, 'C,Q': 3, '0,G': 2, 'F,P': 1, '4,A': 5, '0,D': 9, '0,C': 2, 'G,Q': 1, '2,Z': 9, '5,B': 2, '3,8': 3, '1,4': 1}
{'J,T': 6, '0,3': 2, '1,7': 25, '6,8': 2, '2,9': 1, '3,8': 5, '5,6': 2, 'G,Q': 2, 'K,Y': 1, '9,S': 5, '7,T': 2, '8,B': 8, '9,B': 2, '9,J': 2, '0,Q': 4, 'C,D': 2, '1,T': 20, 'C,G': 1, '6,E': 3, '6,F': 2, '0,D': 37, '0,C': 19, 'E,S': 3}
{'0,4': 1, 'E,F': 1, '6,8': 3, '5,9': 3, 'F,P': 1, '5,6': 8, '1,7': 3, 'G,Q': 1, 'K,X': 1, '4,Q': 1, '9,S': 2, '7,T': 13, '8,B': 36, '9,E': 8, '5,F': 5, '3,B': 2, '0,Q': 52, '1,T': 6, '6,E': 3, 'B,P': 2, '6,G': 2, 'B,R': 2, '0,G': 4, '0,D': 57, '0,C': 19, '0,L': 3}
{'1,7': 3, '6,8': 2, '5,9': 4, 'F,P': 4, '5,6': 4, 'F,J': 2, '4,M': 1, 'N,X': 3, 'M,Y': 7, 'G,U': 2, '7,Z': 1, '9,S': 5, '8,B': 37, '4,A': 1, '9,B': 2, 'R,Z': 2, '5,B': 1, '5,F': 2, '3,B': 2, '0,Q': 5, '1,T': 3, 'C,G': 1, '0,D': 26, '0,C': 10, '2,Z': 1, '1,G': 2, '6,S': 2}
{'J,T': 9, '7,Z': 1, '5,9': 4, '3,8': 3, 'A,K': 7, '0,Q': 8, '1,7': 6, 'C,G': 1, '5,6': 5, '8,B': 13, '0,G': 2, '6,8': 7, '4,A': 7, '0,D': 23, '0,C': 10, 'G,Q': 1, '3,5': 1, 'B,D': 1, 'X,Y': 5, '9,B': 4, '5,8': 5}
{'1,7': 1, '1,3': 1, '6,8': 11, 'P,R': 2, '5,6': 11, 'G,Q': 4, 'K,Y': 1, '9,S': 7, '7,T': 2, '8,B': 71, '5,K': 7, '4,A': 3, '5,F': 2, '3,B': 2, '0,Q': 9, '1,Y': 11, '1,T': 20, '6,B': 6, '6,G': 2, 'D,Q': 2, '0,D': 53, '0,C': 3, '2,Z': 7}
{'J,T': 2, 'C,G': 4, '7,T': 5, '0,Q': 13, '0,1': 2, '5,S': 4, '1,T': 3, '9,S': 6, '5,6': 15, '1,3': 2, '8,B': 13, '9,D': 1, '0,D': 14, '0,C': 1, '4,G': 1, '6,S': 2, '1,C': 1, '4,L': 1}
{'J,T': 2, '0,U': 5, '3,8': 3, '9,S': 2, '0,Q': 4, '7,T': 1, '1,T': 14, '5,6': 19, '8,B': 49, '6,8': 8, '4,A': 4, '0,D': 24, '9,B': 2, '2,Z': 3, '6,S': 2, '3,B': 6}
{'F,Y': 1, 'D,Q': 1, '6,K': 1, '7,T': 1, '6,E': 3, '0,Q': 9, '5,S': 3, '1,T': 3, '1,7': 1, '5,6': 10, '8,B': 27, '4,A': 1, '0,D': 61, '0,C': 25, 'G,Q': 1, '2,Z': 22, '5,B': 5, '3,8': 5, '1,4': 2, '3,B': 1, '9,S': 6}
{'4,7': 1, '9,S': 1, '0,Q': 5, '5,6': 5, '1,T': 7, 'C,G': 2, '7,T': 15, '8,B': 5, '6,8': 2, '0,D': 22, '0,C': 6, '4,Z': 2, 'G,Q': 3, 'C,Q': 2, '3,B': 2}
{'J,T': 2, '1,7': 6, '6,8': 1, '2,3': 1, '5,8': 6, '3,8': 5, '5,6': 109, '4,W': 1, '8,B': 56, '3,S': 5, '4,A': 15, '9,B': 2, '8,S': 29, '3,B': 7, '0,Q': 10, '1,T': 3, '6,E': 12, '0,D': 33, '0,C': 9, '2,Z': 2, '1,G': 4, '6,S': 2, 'C,Q': 2}
{'D,Q': 1, '0,3': 2, 'X,Y': 1, '5,S': 6, '1,7': 7, '1,T': 16, '5,6': 14, '8,B': 35, 'F,P': 1, '6,8': 2, '4,A': 1, '0,D': 10, '0,C': 2, 'B,H': 2, 'G,Q': 1, '2,Z': 3, '5,B': 1, 'A,W': 2, '1,4': 1}
{'J,T': 4, '0,4': 1, '1,7': 2, '6,8': 10, '3,8': 4, '3,9': 1, 'F,P': 5, 'M,N': 5, '1,J': 3, 'M,X': 2, 'K,X': 3, '9,S': 8, '7,T': 2, '8,B': 9, '9,B': 1, '3,B': 2, '5,6': 3, '0,Q': 8, '1,T': 2, '0,G': 1, '0,D': 22, '0,C': 3}
{'J,T': 2, '0,1': 2, '1,7': 4, '3,8': 1, '3,9': 1, '5,6': 2, 'D,P': 7, '9,S': 4, '8,B': 71, '5,T': 4, '4,A': 14, '9,B': 2, '4,F': 1, 'C,L': 3, '0,Q': 1, '1,Y': 1, '1,T': 23, '6,E': 6, '1,C': 2, '0,G': 2, 'D,Q': 1, '0,D': 56, '0,C': 6, '2,Z': 2, '1,F': 2, 'C,Q': 3}
{'7,T': 3, '0,2': 2, '0,Q': 13, '6,E': 1, '1,T': 21, 'C,G': 2, '5,6': 5, '8,B': 14, 'L,U': 3, 'D,Q': 1, '0,D': 4, '0,C': 23, 'X,Y': 1, '2,Z': 19, 'G,Q': 4, '9,S': 4, '3,B': 2, '4,6': 1}
{'J,T': 5, '0,4': 1, '6,8': 9, '9,S': 12, '5,9': 2, '3,8': 2, 'F,P': 2, '5,6': 10, 'G,Q': 1, '3,Y': 1, '8,E': 3, '7,T': 28, '8,B': 25, '9,D': 2, '3,E': 1, '0,Q': 4, '1,T': 20, 'C,G': 1, '6,E': 1, '4,A': 7, '0,D': 60, '0,C': 8, '2,Z': 12, 'B,E': 2}
{'7,T': 5, '3,9': 3, '0,Q': 3, '5,S': 4, '1,T': 3, '6,B': 2, '5,6': 6, '8,B': 9, '6,8': 1, '4,A': 3, '0,D': 33, '4,G': 1, '2,Z': 19, '5,B': 3, 'G,Q': 1, 'K,X': 2, 'K,Y': 1}'''
print(s)

# 收集 混淆集 集合
key_set = []
for item in s.split('\n'):
    a = eval(item)
    key_set += a.keys()

key_set = sorted(set(key_set))
print(key_set)

result = [] 
for item in s.split('\n'):
    a = eval(item)
    for i in key_set:
        if i not in a.keys():
            a[i]=0 
#     print(a)
    
    print([item[1] for item in sorted(a.items(),key = lambda x:x[0])])
    
    result.append(sorted(a.items(),key = lambda x:x[0]))
#     break

# print(result)

## 保存34分类TOP21的应用集预测结果

In [ ]:
%%time

# 34分类前21个模型的预测结果
predict_result_34class_file = open(os.path.join(model_root_path,'34class_o1_predict_result_e5.pkl'),'w')

run_id = [99,129,141,245,249,270,287,300,311,375,425,509,543,630,758,864,875,890,905,975,1014]
# run_id = [875,890,905,975,1014]
run_idx = 0
with open(os.path.join(model_root_path, '34class_model_weight.pkl'), 'r') as fin:
    for index in range(1, len(model_file_list_path) + 1):
        # 从 模型1 开始，依次往后
        # 找到对应模型文件
#         if index != run_id[run_idx]:
#             weights = pickle.load(fin)
#             continue
        
        if index < run_id[run_idx]:
            weights = pickle.load(fin)
            continue

        
        weights = pickle.load(fin)

        # print('OK')
        int_predict,(test_accuracy, graterThan2, graterThan5, graterThan10,badcase) = \
                                            cnn_batch_predict_spark(X_other_rdd,y_other, weights)
        
        print(index,test_accuracy, graterThan2, graterThan5,graterThan10)
        print(badcase)
        
        pickle.dump(np.asarray(int_predict),
                    predict_result_34class_file
                   )

        
#         print(test_accuracy, graterThan5, graterThan10)
        run_idx += 1
        if run_idx>=len(run_id):
            break
#         break


## 保存34分类TOP21的 测试集/应用集 预测结果（用5-6二分类器修正）

In [7]:
%%time
option = 'test'
# option = 'other'

if option == 'test':
    predict_result_34class_file = open(os.path.join(model_root_path, '34class_test_predict_result_e5.pkl'), 'r')
    X = X_test
    y = np.asarray(y_test)
elif option == 'other':
    predict_result_34class_file = open(os.path.join(model_root_path, '34class_other_predict_result_e5.pkl'), 'r')
    X = X_other
    y = y_other

run_id = [99, 129, 141, 245, 249, 270, 287, 300, 311, 375, 425, 509, 543, 630, 758, 864, 875, 890, 905, 975, 1014]
# run_id = [129, 141, 245, 249, 270, 287, 300, 311, 375, 425, 509, 543, 630, 758, 864, 875, 890, 905, 975, 1014]
cou = 0
for index in run_id:
    try:
        int_predict_34class = np.asarray(pickle.load(predict_result_34class_file))
    
    
        # 读取二分类器的权重
        weights = pickle.load(open(os.path.join(model_root_path, '56binary_model_weight.pkl'), 'r'))
        # 被预测成56的数据
        idx_predicted_56 = (int_predict_34class == 5) + (int_predict_34class == 6)
        print('预测成5-6的个数:%d' % sum(idx_predicted_56))


    #     X_rdd = sc.parallelize(X[idx_predicted_56])
    #     X_rdd.cache()

    #     yy = y[idx_predicted_56]

    #     X_rdd.count()
    #     break
    #     binary_result,(test_accuracy, graterThan2, graterThan5, graterThan10,badcase)= cnn_batch_predict_spark(
    #         X_rdd, 
    #         y[idx_predicted_56],
    #         weights
    #     )

    # #     print(index, test_accuracy)
    # #     print(binary_result)
    # #     break
        print(binary_result ==0)
        binary_result = np.asarray(binary_result)
    #     binary_result[binary_result == 0] = 5
    #     binary_result[binary_result == 1] = 6

        # 修正结果
        int_predict_34class[idx_predicted_56] = binary_result
        print(index, count_result(int_predict_34class, y))

        break
        
    except:
        print('end!')

EOFError: 

> /usr/lib/python2.7/pickle.py(880)load_eof()
    878 
    879     def load_eof(self):
--> 880         raise EOFError
    881     dispatch[''] = load_eof
    882 

ipdb> q
